In [20]:
import sys
sys.path.append("..") # go to parent dir to import utilities
import lib_neus_et as neus
import json

import numpy as np
import pandas as pd

In [21]:
data_path= '../data/' #This is the path to recordings

In [22]:
# Load parameters

params = neus.preprocessParameters('../lib_neus_et/parameters/')

Reading parameters


In [23]:
# Load one patient with getUsers function
pat_files = neus.getUsers(data_path, 'example')
print('Num recordings:', len(pat_files))
f = pat_files[0]
f

Num recordings: 1


'../data/001/test-example-sl-0.json'

In [24]:
# Create an instance of ET_Test with the parameters loaded from file
T = neus.ET_Test(f, params)

In [25]:
# Initialize all tasks with parse method. Since it is data from children we use the "meta" parameter.
T.parse(process_mode='meta', verbose=True)

# The end of test was not detected but this is normal with the new tests because they have no final message.

          Test ID: 9
    Eyetracker ID: IS404-101107121473
     Test version: f67cf6e
Screen resolution: 1920x1080

Sorting data.
Checking for back button use.
Parsing data into separate tasks.
  Dummy task found 1x.
End of test detected; data read in full.



In [26]:
# Get sampling rate of the eye-tracking

T.task_list['Dummy'][0]._et.sampling_rate

90

In [27]:
# The list of task objects initialized. One per repetition

T.task_list

{'Dummy': [<lib_neus_et.tasks.dummy.Dummy at 0x2adf842b1f0>]}

In [28]:
# Compute all features
_ = T.compute(task_list=None,verbose=True)


Computing features for task Dummy.

Computing features of Dummy task.


In [29]:
# Now features are available 
T.features

{'Dummy': [{'dummy.nan': {'L': 0.4961133973479652, 'R': 0.5016003657978967, 'M': 0.4897119341563786, 'join': nan}, 'dummy.fix_num': {'L': 20, 'R': 19, 'M': 17, 'join': nan}, 'dummy.cat': {'L': nan, 'R': nan, 'M': nan, 'join': 'A'}}]}

In [30]:
# We can reduce to a single value each feature (collapse multiple eyes by taking the best). This must be ALWAYS done after computing all features.
_ = T.compute_best_eye(verbose=True)

Computing features for task Dummy.


In [31]:
# And then the output can be accessed separately
T.features_be

{'Dummy': [{'dummy.nan': 0.4897119341563786, 'dummy.fix_num': 19, 'dummy.cat': 'A'}]}

In [32]:
# Create a pandas DataFrame 

df = T.to_pandas(best_eye=False)

In [33]:
df.head()

,num_rep,dummy.nan.L,dummy.nan.R,dummy.nan.M,dummy.nan.join,dummy.fix_num.L,dummy.fix_num.R,dummy.fix_num.M,dummy.fix_num.join,dummy.cat.L,dummy.cat.R,dummy.cat.M,dummy.cat.join
0,0,0.496113,0.5016,0.489712,NaN,20,19,17,NaN,NaN,NaN,NaN,A


In [34]:
# Same can be done with best eye

df = T.to_pandas(best_eye=True)
df.head()

,num_rep,dummy.nan,dummy.fix_num,dummy.cat
0,0,0.489712,19,A


In [35]:
# Let's now introduce a threshold of missing value and remove some repetitions (removed repetitions are lost and must be computed again)

print('Number of repetitions before:', len(T.features['Dummy']))

T.threshold_nan('Dummy', 'M', threshold=0.1)

print('Number of repetitions after:',len(T.features['Dummy']))

Number of repetitions before: 1
Number of repetitions after: 0


The task was removed because it has more than 10% of missing values, leaving the dataset empty.